<h1>Содержание<span class="tocSkip"></span></h1>
<li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Нам необходимо создать инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучим модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

## Подготовка

In [2]:
# подключим все необходимые библиотеки


import pandas as pd
import re
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
import spacy
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Загрузим и изучим данные.

In [4]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [5]:
data.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Данные изучили, переходим к предобработке. Лемматезируем текст каждого комментария и избавимся от лишних символов.

Почистим текст от стоп слов, уберём регистр и лишние знаки.

In [7]:
data['text'] = data['text'].str.lower()

In [8]:
# преобразуем столбец text в список текстов
corpus = list(data['text'])

In [9]:
# избавимся от лишних символов и знаков препинания
def clear_text(row):
    text = re.sub(r"[^a-zA-Z']", ' ', row)
    text = ' '.join(text.split())
    return text

In [10]:
for i in range(len(corpus)):
    corpus[i] = clear_text(corpus[i])

In [10]:
%%time 

lemmatized_text = []
for i in range(len(corpus)):
    doc = nlp(corpus[i])
    lemm_sentence = ''
    
    for token in doc:
        if str(token.lemma_) != '-PRON-':
            lemm_sentence+=str(token.lemma_)
            lemm_sentence+=' '
    lemmatized_text.append(lemm_sentence)

CPU times: user 35min 38s, sys: 34.6 s, total: 36min 13s
Wall time: 36min 31s


In [11]:
data['lemmatized_text'] = lemmatized_text

In [18]:
data.to_csv('out', index=False)

In [11]:
data = pd.read_csv('out')

lemmatized_text = list(data['lemmatized_text'])

### Вывод

Загрузили и изучили данные. Далее перешли к их предобработке. Комментарии привели к нижнему регистру, очистили текст от лишних символов, привели слова к начальной форме. Далее переходим к обучению моделей.

## Обучение

В данной части нашей работы обучим разные модели и рассчитаем качество каждой метрикой F1.

Поделим выборки на тренировочную и тестовую. Для определения тональности применим величины TF-IDF как признаки.

In [18]:
train_corpus, temporary_corpus, target_train, target_temporary = train_test_split(
    data['lemmatized_text'], data['toxic'], test_size=0.4, random_state=300500)

valid_corpus, test_corpus, target_valid, target_test = train_test_split(
    temporary_corpus, target_temporary, test_size=0.5, random_state=300500)

In [19]:
print(target_train.shape, target_test.shape, target_valid.shape)

(95742,) (31915,) (31914,)


In [21]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [22]:
train_tf_idf = count_tf_idf.fit_transform(train_corpus.values.astype('U'))
valid_tf_idf = count_tf_idf.transform(valid_corpus.values.astype('U'))
test_tf_idf = count_tf_idf.transform(test_corpus.values.astype('U'))

Далее переходим к оценке тональности. То есть построим модель и спрогнозируем результат на тестовой выборке, затем рассчитаем качество модели при помощи метрики F1_score. Для прогнозирования попробуем 3 модели:
* LogisticRegression
* DecisionTreeClassifier
* RandomForestClassifier

In [23]:
model = LogisticRegression(random_state=30050).fit(train_tf_idf, target_train)
predictions = list(model.predict(valid_tf_idf))
print(f1_score(target_valid, predictions))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7204077779875402


Попробуем улучшить метрику качества, изменив пороговое значение.

In [46]:
best_f1 = 0
best_threshold = 0
model = LogisticRegression(random_state=30050).fit(train_tf_idf, target_train)
probabilities_valid = model.predict_proba(valid_tf_idf)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0, 0.4, 0.02):
    predicted_valid = probabilities_one_valid > threshold
    if f1_score(target_valid, predicted_valid) > best_f1:
        best_f1 = f1_score(target_valid, predicted_valid)
        best_threshold = threshold
print(f'Best F1-score = {best_f1}, threshold = {threshold}')

Best F1-score = 0.7711386696730554, threshold = 0.38


Логистическая регрессия показала значение метрики f1_score равное 0.73, изменив значение порога на 0.38 удалось достичь значение метрики качества равного 0.73. Далее посмотрим, как покажет себя модель DecisionTree.

In [25]:
model = DecisionTreeClassifier(random_state=30050).fit(train_tf_idf, target_train)
predictions = list(model.predict(valid_tf_idf))
print(f1_score(target_valid, predictions))

0.7052779075198505


In [26]:
best_f1 = 0
best_depth = 0

for depth in [1, 10, 100, 200]:
    model = DecisionTreeClassifier(random_state=30050, max_depth=depth).fit(train_tf_idf, target_train)
    predictions = list(model.predict(valid_tf_idf))
    if f1_score(target_valid, predictions) > best_f1:
        best_f1 = f1_score(target_valid, predicted_valid)
        best_depth = depth
print(f'Best F1-score = {best_f1}, max_depth = {depth}')

Best F1-score = 0.6943086250733425, max_depth = 200


Использую модель DecisionTree, лучшее значение метрики F1, которое удалось получить равно 0.7, что меньше чем у LogisticRegression. Далее посмотрим, как покажет себя RandomForestClassifier.

In [27]:
model = RandomForestClassifier(random_state=30050).fit(train_tf_idf, target_train)
predictions = list(model.predict(valid_tf_idf))
print(f1_score(target_valid, predictions))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.654639175257732


In [30]:
best_f1 = 0
best_est = 0
for n_est in [10, 50, 100]:
    model = RandomForestClassifier(random_state=30050, n_estimators=n_est).fit(train_tf_idf, target_train)
    predictions = list(model.predict(valid_tf_idf))
    
    if f1_score(target_valid, predictions) > best_f1:
        best_f1 = f1_score(target_valid, predicted_valid)
        best_est = n_est
print(f'Best F1-score = {best_f1}, n_estimators = {n_est}')

Best F1-score = 0.6943086250733425, n_estimators = 100


In [29]:
best_f1 = 0
best_depth = 0

for depth in [10, 100, 500]:
    model = RandomForestClassifier(random_state=30050, n_estimators=50, max_depth=depth).fit(train_tf_idf, target_train)
    predictions = list(model.predict(valid_tf_idf))
    if f1_score(target_valid, predictions) > best_f1:
        best_f1 = f1_score(target_valid, predicted_valid)
        best_depth = depth
print(f'Best F1-score = {best_f1}, max_depth = {depth}')

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Best F1-score = 0.6943086250733425, max_depth = 500


Использую модель RandomForestClassifier, лучшее значение метрики F1, которое удалось получить равно 0.69 (если увеличить значения гиперпараметров, возможно, модель покажет лучшее значения, но тогда она будет обучаться долго), что меньше чем у LogisticRegression.
Таким образом, LogisticRegression показала себя лучше всех. Проверим её на тестовой выборке, посмотрим, какое значение метрики f1 покажет модель.

In [49]:
threshold = 0.38

model = LogisticRegression(random_state=30050).fit(train_tf_idf, target_train)
probabilities_test = model.predict_proba(test_tf_idf)
probabilities_one_test = probabilities_test[:, 1]

predicted_test = probabilities_one_test > threshold

print(f'F1 score = {f1_score(target_test, predicted_test)}')

F1 score = 0.7535528596187175


## Выводы

В данной работе мы подбирали лучшую модель для классификации комментариев на позитивные и негативные. Мы подготовили наши данные, лемматизировали их. Затем подготовили наши признаки, для оценки важности слов использовали величину TF-IDF. Нашей задачей была: Построить модель со значением метрики качества F1 не меньше 0.75. Нам удалось её выполнить, мы воспользовались тремя моделями: LogisticRegression, DecisionTreeClassifier, RandomForestClassifier. Лучше всех себя показала логистическая регрессия, изменив пороговое значение для предсказывания на 0.38. Удалось получить значение метрики F1 0.753. таким образом, интернет-магазину «Викишоп» для классификации тональности комментариев советую использовать модель логистической регрессии с пороговым значением равным 0.38.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны